**Group Name:** \_\_\_\_\_

**Members:** \_\_\_\_\_

## Template of the Test Code
The test code template is shown below:

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"


# ===============================
# Test Dataset
# ===============================
class TestDataset(Dataset):
    def __init__(self, test_dir, csv_path):
        """
        test_dir: path to test images folder
        csv_path: path to CSV file containing 'ID' and 'label' columns
        """
        self.data = pd.read_csv(csv_path, dtype={'ID': str})
        self.test_dir = test_dir
        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = str(self.data.iloc[idx]['ID'])
        label = int(self.data.iloc[idx]['label'])

        img_path = os.path.join(self.test_dir, img_id + ".jpg")
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, label


# ===============================
# CNN Model
# ===============================
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ===============================
# Test Function
# ===============================
def test_model():
    data_root = "path_to_dataset"
    model_path = "model.pth"

    test_dir = os.path.join(data_root, "test")
    csv_path = "path_to_csv"

    test_dataset = TestDataset(test_dir, csv_path)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = CNN().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    all_labels, all_preds = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Testing"):
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    acc = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item()
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    final_score = 0.3 * precision + 0.3 * recall + 0.4 * f1

    print("===== Test Results =====")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1:.4f}")
    print(f"Final Score : {final_score:.4f}")


if __name__ == "__main__":
    test_model()


## What TAs do
In this way, TAs can get the test result by only replacing the "data_root" and "model_path".

In [ ]:
data_root = "path_to_dataset"
model_path = "saved_model.pth"

0.5